In [1]:
import pyworld as pw
from scipy.io import wavfile
import numpy as np
import pysptk
import matplotlib.pyplot as plt
import os
from glob import glob
from dtw import dtw
from numpy.linalg import norm
import chainer
from chainer import Variable, serializers, Chain, cuda,optimizers
import chainer.functions as F
import chainer.links as L

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
source="rms"
target="slt"

In [3]:
source_path_list=glob(os.path.join("","train/"+source+"/*.wav"))
target_path_list=glob(os.path.join("","train/"+target+"/*.wav"))
source_fs=[]#入力のサンプリング周波数
source_data=[]#入力のデータ
target_fs=[]#出力のサンプリング周波数
target_data=[]#出力のデータ
source_raw_f0=[]#基本周波数
target_raw_f0=[]
source_t=[]
target_t=[]
source_f0=[]#修正された基本周波数
target_f0=[]
source_sp=[]#スペクトル包絡
target_sp=[]
source_ap=[]#非周期成分
target_ap=[]
source_mc=[]#MCEP
target_mc=[]
source_aligned_mc=[]#同期されたMCEP
target_aligned_mc=[]

#for i in range(len(source_path_list)):#データの読み込み
for i in range(100):
    source_fs.append(wavfile.read(source_path_list[i])[0])
    source_data.append(wavfile.read(source_path_list[i])[1])
    target_fs.append(wavfile.read(target_path_list[i])[0])
    target_data.append(wavfile.read(target_path_list[i])[1])
    source_data[i]=source_data[i].astype(np.float)
    target_data[i]=target_data[i].astype(np.float)

print("Data Loaded")
    
#for i in range(len(source_path_list)):#各特徴量の抽出
for i in range(100):
    source_raw_f0.append(pw.dio(source_data[i],source_fs[i])[0])
    source_t.append(pw.dio(source_data[i],source_fs[i])[1])
    source_f0.append(pw.stonemask(source_data[i],source_raw_f0[i],source_t[i],source_fs[i]))
    source_sp.append(pw.cheaptrick(source_data[i],source_f0[i],source_t[i],source_fs[i]))
    source_ap.append(pw.d4c(source_data[i],source_f0[i],source_t[i],source_fs[i]))
    source_mc.append(pysptk.sp2mc(source_sp[i],24,pysptk.util.mcepalpha(source_fs[i])))
    target_raw_f0.append(pw.dio(target_data[i],target_fs[i])[0])
    target_t.append(pw.dio(target_data[i],target_fs[i])[1])
    target_f0.append(pw.stonemask(target_data[i],target_raw_f0[i],target_t[i],target_fs[i]))
    target_sp.append(pw.cheaptrick(target_data[i],target_f0[i],target_t[i],target_fs[i]))
    target_ap.append(pw.d4c(target_data[i],target_f0[i],target_t[i],target_fs[i]))
    target_mc.append(pysptk.sp2mc(target_sp[i],24,pysptk.util.mcepalpha(target_fs[i])))
    
print("Features Extracted")
    
#for i in range(len(source_path_list)):#dtwを用いて同期化
for i in range(100):
    dist,cost,acc_cost,path=dtw(source_mc[i],target_mc[i],dist=lambda x, y: norm(x - y, ord=1))
    source_aligned_mc.append(source_mc[i][path[0]])
    target_aligned_mc.append(target_mc[i][path[1]])
    
print("Time Wrapped")


Data Loaded
Features Extracted
Time Wrapped


In [5]:
gpu_id = 0

if gpu_id is not None:
    xp = cuda.cupy
else:
    xp = np

class VCBLSTM(chainer.Chain):
    
    def __init__(self,io_size=25,layer1_size=128,layer2_size=256):
        super(VCBLSTM,self).__init__(
            Linear_input=L.Linear(io_size,layer1_size),
            #BLSTM1=L.NStepBiLSTM(1,layer1_size,layer2_size,0.5),
            #BLSTM2=L.NStepBiLSTM(1,layer2_size,layer1_size,0.5),
            BLSTM=L.NStepBiLSTM(2,128,128,0.5),
            #Linear_output=L.Linear(layer1_size,io_size),
            Linear_output=L.Linear(256,25)
        )

        #self.reset_state()

        #if gpu_id is not None:
        #    cuda.get_device(gpu_id).use()
        #    self.to_gpu(gpu_id)
            
    def __call__(self,source):
        hx=None
        cx=None
        Lin=self.Linear_input(Variable(xp.array(source,dtype=xp.float32)))
        #hx1,cx1,ys1 = self.BLSTM1(hx,cx,Lin)
        #hx2,cx2,ys2 = self.BLSTM2(hs,cx,ys1)
        xs=[]
        xs.append(Lin)
        hx2,cx2,ys2=self.BLSTM(hx,cx,xs)
        #Lout=self.Linear_output(ys2)
        Lout=self.Linear_output(ys2[0])
        return Lout
        
#    def reset_state(self):
#        hx1=None
#        hx2=None
#        cx1=None
#        cx2=None

    def loss(self,source,target):
        y=self(source)
        return F.mean_squared_error(y,target)
    
    def load_model(self,file_name):
        serializers.load_npz(file_name, self)
    
    def save_model(self,file_name):
        serializers.save_npz(file_name,self)

In [7]:
source_list=[]
target_list=[]
#for i in range(len(source_path_list)):
for i in range(100):
    source_list.append(xp.array(source_aligned_mc[i],dtype=xp.float32))
    target_list.append(xp.array(target_aligned_mc[i],dtype=xp.float32))

In [8]:
model=VCBLSTM()
optimizer=optimizers.Adam()
optimizer.setup(model)

cuda.get_device(gpu_id).use()
model.to_gpu(gpu_id)


epoch_num=50
for epoch in range(epoch_num):
    print("{0} / {1} epoch start.".format(epoch + 1, epoch_num))
    acc_loss=0.0
    for i in range(100):
        loss=0.0
        #model.reset_state()
        model.cleargrads()
        loss=model.loss(source_list[i],target_list[i])
        acc_loss+=loss.data
        loss.backward()
        #loss.unchain_backward()
        optimizer.update()
    model_file="trained_model/VCBLSTM_"+str(epoch)+".model"
    model.save_model(model_file)
    print("epoch {0}:loss = {1}.".format(epoch+1,acc_loss))
#model.to_cpu()
model_file="trained_model/VCBLSTM.model"
model.save_model(model_file)
    

1 / 50 epoch start.
epoch 1:loss = 17.809664.
2 / 50 epoch start.
epoch 2:loss = 9.395895.
3 / 50 epoch start.
epoch 3:loss = 8.548403.
4 / 50 epoch start.
epoch 4:loss = 7.857564.
5 / 50 epoch start.
epoch 5:loss = 7.362438.
6 / 50 epoch start.
epoch 6:loss = 7.016853.
7 / 50 epoch start.
epoch 7:loss = 6.7792945.
8 / 50 epoch start.
epoch 8:loss = 6.5709925.
9 / 50 epoch start.
epoch 9:loss = 6.417126.
10 / 50 epoch start.
epoch 10:loss = 6.288497.
11 / 50 epoch start.
epoch 11:loss = 6.1827216.
12 / 50 epoch start.
epoch 12:loss = 6.092363.
13 / 50 epoch start.
epoch 13:loss = 6.0117836.
14 / 50 epoch start.
epoch 14:loss = 5.9570127.
15 / 50 epoch start.
epoch 15:loss = 5.8680167.
16 / 50 epoch start.
epoch 16:loss = 5.7686653.
17 / 50 epoch start.
epoch 17:loss = 5.7319865.
18 / 50 epoch start.
epoch 18:loss = 5.671283.
19 / 50 epoch start.
epoch 19:loss = 5.55146.
20 / 50 epoch start.
epoch 20:loss = 5.4341483.
21 / 50 epoch start.
epoch 21:loss = 5.3799024.
22 / 50 epoch start.


In [35]:
source_test_path_list=glob(os.path.join("","test/"+source+"/*.wav"))
target_test_path_list=glob(os.path.join("","test/"+target+"/*.wav"))

In [36]:
source_test_fs=[]#入力のサンプリング周波数
source_test_data=[]#入力のデータ
target_test_fs=[]#出力のサンプリング周波数
target_test_data=[]#出力のデータ
source_test_raw_f0=[]#基本周波数
target_test_raw_f0=[]
source_test_t=[]
target_test_t=[]
source_test_f0=[]#修正された基本周波数
target_test_f0=[]
source_test_sp=[]#スペクトル包絡
target_test_sp=[]
source_test_ap=[]#非周期成分
target_test_ap=[]
source_test_mc=[]#MCEP
target_test_mc=[]
source_test_aligned_mc=[]#同期されたMCEP
target_test_aligned_mc=[]

for i in range(5):
    source_test_fs.append(wavfile.read(source_test_path_list[i])[0])
    source_test_data.append(wavfile.read(source_test_path_list[i])[1])
    target_test_fs.append(wavfile.read(target_test_path_list[i])[0])
    target_test_data.append(wavfile.read(target_test_path_list[i])[1])
    source_test_data[i]=source_test_data[i].astype(np.float)
    target_test_data[i]=target_test_data[i].astype(np.float)
    
#for i in range(len(source_path_list)):#各特徴量の抽出
for i in range(5):
    source_test_raw_f0.append(pw.dio(source_test_data[i],source_test_fs[i])[0])
    source_test_t.append(pw.dio(source_test_data[i],source_test_fs[i])[1])
    source_test_f0.append(pw.stonemask(source_test_data[i],source_test_raw_f0[i],source_test_t[i],source_test_fs[i]))
    source_test_sp.append(pw.cheaptrick(source_test_data[i],source_test_f0[i],source_test_t[i],source_test_fs[i]))
    source_test_ap.append(pw.d4c(source_test_data[i],source_test_f0[i],source_test_t[i],source_test_fs[i]))
    source_test_mc.append(pysptk.sp2mc(source_test_sp[i],24,pysptk.util.mcepalpha(source_test_fs[i])))
    target_test_raw_f0.append(pw.dio(target_test_data[i],target_test_fs[i])[0])
    target_test_t.append(pw.dio(target_test_data[i],target_test_fs[i])[1])
    target_test_f0.append(pw.stonemask(target_test_data[i],target_test_raw_f0[i],target_test_t[i],target_test_fs[i]))
    target_test_sp.append(pw.cheaptrick(target_test_data[i],target_test_f0[i],target_test_t[i],target_test_fs[i]))
    target_test_ap.append(pw.d4c(target_test_data[i],target_test_f0[i],target_test_t[i],target_test_fs[i]))
    target_test_mc.append(pysptk.sp2mc(target_test_sp[i],24,pysptk.util.mcepalpha(target_test_fs[i])))
    

In [38]:
source_test_list=[]
target_test_list=[]
#for i in range(len(source_path_list)):
for i in range(5):
    source_test_list.append(xp.array(source_test_mc[i],dtype=xp.float32))
    target_test_list.append(xp.array(target_test_mc[i],dtype=xp.float32))

In [39]:
tgt=model(source_test_list[0]).data.astype(np.float64)
tgt=xp.asnumpy(tgt)

In [40]:
sp=pysptk.mc2sp(tgt,0.42,512*2)
owav=pw.synthesize(source_test_f0[0],sp,source_test_ap[0],source_test_fs[0])
owav=np.clip(owav,-32768,32767)
wavfile.write("test.wav",source_test_fs[0],owav.astype(np.int16))